# Analysis of Brandeis Computer Science classes
Here we show how to use pandas to analyze the Brandeis course data stored in data/courses

Some questions we can try to answer
* which faculty have taught the most students over the Fall2004-Summer2021 interval
* which departments offer the most courses
* which departments have had the most students
* how have the enrollents by departments changed
* etc, as in PA01

We'll show how to use pivot tables and aggregation to answer some of these.

This notebook shows how to use aggregation with groupby, pivot tables, and how to create new columns by applying some function to other columns.


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

First we read in the data from Fall 2004 to Summer 2021 with minimal info about each course

In [ ]:
df = pd.read_csv('data2/classes/courses.csv')
df

In [ ]:
cs = df[df['subj']=='COSI'].copy()
cs

In [ ]:
cs['inst'] = cs['inst'].apply(lambda x: "Hickey, Timothy" if x=="Hickey, Timothy J" else x)

# Finding teachers with the most students
The easiest way to do this is with a groupby it shows the three stages of groupby
1. grouping of rows together based on the value in a column ('inst')
1. projection of those rows into a set of values (what value are we interested in) (enr)
1. aggregation of those values into a single value with some operator from arrays to numbers (sum)

In [ ]:
#fac = df.groupby(['inst','subject'])['enr'].agg(sum)
#fac
fac = cs.groupby(['inst','subj'])
fac = fac['enr'].agg([sum, np.mean, np.max,len])
fac = fac.sort_values(ascending=False,by='len')
fac.head(20)



## Sorting the Series
We can sort the values if we want (and here I am then reversing them by iterating with a step of -1 through the array) and we can use .head(N) and .tail(N) to look at either end of the sorted series.

In [ ]:
fac = fac.sort_values(ascending=False,by='sum')
fac.head(20)


In [ ]:
fac.head(50)

In [ ]:
df

# Restricting to a time interval
Let's do the same but just look for the past 4 years, i.e. from Fall2017 to Summer 2021 which are the codes 1173 to 1212.


In [ ]:
df2 = cs[cs['code']>=1193]
df2
#islater = df['code']>= 1173
#df[islater]

## aggregate by summing the enrollments for each subject

In [ ]:
insts = df2.groupby(['inst'])['enr'].agg(sum)
insts = insts.sort_values(ascending=False)
insts.head(20)
                     


# Time series
Now lets try to get time series data and plot the enrollments in a department by year.
Let's first add a new column giving the academic year, so Fall2004 and Spring 2005 and Summer2005 would all be 2004


In [ ]:
def code2year(code):
    ''' returns the academic year corresponding to a code'''
    code=str(code)
    year = int("20"+code[1:-1])
    if code[-1]!='3':
        year = year -1
    return year
print(code2year('1051'))



### the original data frame has no 'year' column

In [ ]:
df

### create a new column for the year and the courseid
We can apply the code2year function to each of the integers in the 'code' column to get a new series,
and then assign that series to be a new column 'year' of the df dataframe

In [ ]:
cs['year'] = cs['code'].apply(code2year)
cs

In [ ]:
cs['courseid'] = cs['num'].apply(str)+cs['suf']
cs

In [ ]:
cs

# Group by the year and subject
This creates a new Series with a multi-index containing both year and subject

In [ ]:
cs_enr = cs.groupby('year')['enr'].agg(sum)
cs_enr.plot()
plt.axis([2004,2021,0,2500])
plt.grid()


In [ ]:
classes_enr = cs.groupby(['courseid','year'])['enr'].agg(sum)
classes_enr

In [ ]:
classes_enr = classes_enr.reset_index(level=['courseid'])
classes_enr

# Create a Pivot table of enrollments by classes and year
Now we do the usual construction of a pivot table

In [ ]:
classes_pt =  pd.pivot_table(classes_enr, values='enr', index=["year"], columns=["courseid"])
classes_pt

classes_pt['CS0'] = classes_pt['2A'].fillna(0)+ classes_pt['10A'].fillna(0)+ classes_pt['11A'].fillna(0)# Plot selected columns of the pivot table
We can now select a few columns and plot them!

In [ ]:
classes_pt['CS0'] = classes_pt['2A'].fillna(0)+ classes_pt['10A'].fillna(0)+ classes_pt['11A'].fillna(0)
classes_pt['31'] = classes_pt['31A'].fillna(0) + classes_pt['131A'].fillna(0)
classes_pt['30'] = classes_pt['30A'].fillna(0) + classes_pt['130A'].fillna(0)
classes_pt['64'] = classes_pt['65A'].fillna(0) + classes_pt['164A'].fillna(0)

In [ ]:
classes_pt[['CS0','12B','21A']].plot(figsize=(15,10))
plt.legend()
plt.grid()
plt.title('Total number of students who have taken classes, by year')
plt.savefig('images/introcs.png')

In [ ]:
classes_pt[['CS0','12B','21A','29A','31','30','64']].cumsum().plot(figsize=(15,10))
plt.legend()
plt.grid()
plt.title('Total number of students who have taken classes, by year')

## Most popular Computer Science courses at Brandeis

In [ ]:
classes_pt

In [ ]:
df2 = cs[cs['code']>1042]
courses = df2.groupby(['courseid','title'])['enr'].agg(sum)
courses.sort_values(inplace=True, ascending=False)
courses

In [ ]:
fac = cs.groupby('courseid')
fac = fac['enr'].agg([sum])
fac = fac.sort_values(ascending=False, by='sum').head(40)
fac

In [ ]:
pc1 = cs.groupby('courseid')['inst'].agg(list).sort_values(ascending=False)
pc2 = pc1.apply(lambda x: len(x))
pc = pd.DataFrame([pc1,pc2])
pc